Al momento hai:

Un indice TF-IDF sui profili testuali (text_profile) dei giocatori.

Uno strato “intelligente” di query (Step C) che:

interpreta ruolo (PG/SG/SF/PF/C),

riconosce skill (tiratore da 3, rimbalzista offensivo, difensore, playmaker, scorer…),

costruisce una query pesata e fa ranking con similarità coseno.

👉 Con lo Step D vogliamo:

Tenere questo ranking,

ma aggiungere un livello di spiegabilità:

per ogni giocatore top-N, dire perché è compatibile con la richiesta, usando:

ruolo,

statistiche (3P%, FT%, TRB, ORB, AST, DWS, PTS, …),

skill richieste nella query.

In [22]:
import os
import re
import numpy as np
import pandas as pd
import joblib

from sklearn.metrics.pairwise import cosine_similarity

# ============================
# 1. Caricamento indice + dataset stats
# ============================

MODELS_DIR = "../data/models"
DATA_CLEAN_PATH = "../data/drafted_cleaned.csv"

VECTORIZER_PATH = os.path.join(MODELS_DIR, "tfidf_vectorizer.joblib")
MATRIX_PATH = os.path.join(MODELS_DIR, "tfidf_matrix.joblib")
META_PATH = os.path.join(MODELS_DIR, "index_metadata.csv")

vectorizer = joblib.load(VECTORIZER_PATH)
tfidf_matrix = joblib.load(MATRIX_PATH)
metadata_df = pd.read_csv(META_PATH)

print("Indice caricato.")
print("TF-IDF matrix shape:", tfidf_matrix.shape)
print("Metadata shape:", metadata_df.shape)

# Carico dataset completo con le stats per spiegazioni
full_df = pd.read_csv(DATA_CLEAN_PATH)
print("Dataset stats shape:", full_df.shape)

# Allineo un id univoco condiviso se non esiste già
if "player_id" not in metadata_df.columns:
    metadata_df["player_id"] = metadata_df.index.astype(int)



# ============================
# 2. Utility di base
# ============================

def normalize_text(text: str) -> str:
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = re.sub(r"\s+", " ", text).strip()
    return text

def safe_get(row, col, default=None):
    if col in row.index and not pd.isna(row[col]):
        return row[col]
    return default

def format_percent_it(value, decimals=1):
    """
    Converte 0.384 -> '38,4%' e 38.4 -> '38,4%'.
    """
    if value is None or pd.isna(value):
        return None
    try:
        v = float(value)
    except (TypeError, ValueError):
        return None
    if v <= 1:
        v *= 100
    s = f"{v:.{decimals}f}"
    s = s.replace(".", ",")  # formattazione "italiana"
    return s + "%"

def format_int_it(value):
    """
    Converte un numero in stringa con separatore migliaia stile '10.532'.
    """
    if value is None or pd.isna(value):
        return None
    try:
        v = int(round(float(value)))
    except (TypeError, ValueError):
        return None
    s = f"{v:,}".replace(",", ".")
    return s

# ============================
# 3. Stesse regole semantiche Step C
# ============================

# Skill principali: tiri, rimbalzi, difesa, playmaking
SKILL_PATTERNS = {
    "shooting_3": [
        "tiratore da 3", "ottime percentuali da 3", "buon tiratore da 3",
        "da 3 punti", "tripla", "3 punti"
    ],
    "ft": [
        "buon ft", "ottimo ft", "buon tiratore di liberi", "ottimo tiratore di liberi",
        "buon ft%", "buon ft", "tiri liberi", "liberi"
    ],
    "reb": [
        "rimbalzista", "forte rimbalzista", "buon rimbalzista", "rimbalzi",
        "dominante a rimbalzo", "forte a rimbalzo"
    ],
    "playmaking": [
        "playmaker", "buon passatore", "assist", "creatore di gioco",
        "regista", "fa girare la squadra"
    ],
    "scorer": [
        "realizzatore", "scorer", "tanti punti", "tanti punti a partita",
        "prima opzione offensiva"
    ]
}

# Espansione di testo che useremo per ogni concetto (parole che esistono nei profili)
SEMANTIC_EXPANSIONS = {
    "shooting_3": "ottimo tiratore da 3 3 punti 3P% buon tiratore da 3",
    "ft": "tiri liberi FT% buon tiratore ai liberi eccellente tiratore ai liberi",
    "reb": "rimbalzi forte rimbalzista buon rimbalzista",
    "playmaking": "assist buon passatore ottimo playmaker",
    "scorer": "segna molti punti realizzatore punti a partita"
}

# Peso (quante volte ripetere l’espansione nel testo pesato)
SEMANTIC_WEIGHTS = {
    "shooting_3": 4,
    "ft": 3,
    "reb": 3,
    "playmaking": 3,
    "scorer": 3
}

#Status del giocatore, se è in attività o meno
STATUS_PATTERNS = {
    "Active": ["in attività", "ancora in attività", "ancora gioca", "sta ancora giocando"],
    "Retired": ["ritirato", "non gioca più", "ha smesso", "in pensione"]
}
# ============================
# 4. Parsing query -> intenti
# ============================

def parse_query_intents(raw_query: str):
    q = normalize_text(raw_query)
    detected_status = None
    detected_skills = set()

    # Stato (attivo / ritirato)
    for status, patterns in STATUS_PATTERNS.items():
        for p in patterns:
            if p in q:
                detected_status = status
                break
        if detected_status is not None:
            break
    # Skill
    for skill, patterns in SKILL_PATTERNS.items():
        for p in patterns:
            if p in q:
                detected_skills.add(skill)
                break

    intents = {
        "normalized_query": q,
        "skills": sorted(list(detected_skills)),
        "status": detected_status if detected_status is not None else None

    }
    return intents


def build_weighted_query(raw_query: str, intents: dict) -> str:
    base = intents.get("normalized_query", normalize_text(raw_query))
    boosted_parts = [base]

    for skill in intents.get("skills", []):
        if skill in SEMANTIC_EXPANSIONS:
            exp = SEMANTIC_EXPANSIONS[skill]
            w = SEMANTIC_WEIGHTS.get(skill, 1)
            boosted_parts.append((" " + exp) * w)

    weighted_query = " ".join(boosted_parts)
    return weighted_query.strip()

# ============================
# 5. Ricerca con query pesata (come Step C)
# ============================

def search_players_scout(query: str, top_k: int = 10):
    """
    Versione avanzata della search:
    - parse semantico della query (skills + status)
    - costruzione query pesata
    - similarità coseno contro i profili indicizzati
    - restituisce un DataFrame con similarity + attrs di debug
    """
    if not query or not isinstance(query, str):
        raise ValueError("La query deve essere una stringa non vuota.")

    # 1) Parsing semantico
    intents = parse_query_intents(query)

    # 2) Costruzione query pesata
    weighted_query = build_weighted_query(query, intents)

    # 3) TF-IDF della query
    query_vec = vectorizer.transform([weighted_query])

    # 4) Similarità coseno
    sim_scores = cosine_similarity(query_vec, tfidf_matrix).ravel()
    top_idx = np.argsort(sim_scores)[::-1][:top_k]

    results = metadata_df.iloc[top_idx].copy()
    results["similarity"] = sim_scores[top_idx]

    # Salvo info utili
    results.attrs["intents"] = intents
    results.attrs["weighted_query"] = weighted_query

    return results

# ============================
# 6. Funzioni per spiegazioni 
# ============================

def bucketize_3p(pct):
    """Ritorna un aggettivo qualitativo per il tiro da 3."""
    if pct is None:
        return None
    try:
        v = float(pct)
    except (TypeError, ValueError):
        return None
    if v <= 1:
        v = v * 100
    if v >= 40:
        return "ottimo tiratore da 3 punti"
    elif v >= 35:
        return "buon tiratore da 3 punti"
    elif v >= 30:
        return "tiratore da 3 punti discreto"
    else:
        return "tiratore da 3 punti poco affidabile"

def bucketize_ft(pct):
    """Aggettivo per i tiri liberi."""
    if pct is None:
        return None
    try:
        v = float(pct)
    except (TypeError, ValueError):
        return None
    if v <= 1:
        v = v * 100
    if v >= 85:
        return "eccellente tiratore di tiri liberi"
    elif v >= 75:
        return "buon tiratore di tiri liberi"
    else:
        return "tiratore di liberi migliorabile"

def bucketize_reb(reb):
    """Aggettivo per rimbalzi totali."""
    if reb is None:
        return None
    try:
        v = float(reb)
    except (TypeError, ValueError):
        return None
    if v >= 10:
        return "forte rimbalzista"
    elif v >= 7:
        return "buon rimbalzista"
    elif v >= 4:
        return "rimbalzista discreto"
    else:
        return "rimbalzista non di impatto"

def bucketize_ast(ast):
    """Aggettivo per assist."""
    if ast is None:
        return None
    try:
        v = float(ast)
    except (TypeError, ValueError):
        return None
    if v >= 9:
        return "ottimo playmaker e passatore"
    elif v >= 6:
        return "buon passatore"
    else:
        return "non particolarmente orientato agli assist"

def bucketize_scorer(pts):
    """Aggettivo per capacità realizzativa (punti a partita)."""
    if pts is None:
        return None
    try:
        v = float(pts)
    except (TypeError, ValueError):
        return None
    if v >= 25:
        return "realizzatore di altissimo livello"
    elif v >= 18:
        return "buon realizzatore"
    elif v >= 12:
        return "realizzatore discreto"
    else:
        return "giocatore non focalizzato sulla realizzazione"

def build_explanation(player_row, intents):
    """
    Costruisce una breve spiegazione del perché il giocatore matcha la query.
    Usa:
      - stato (Status)
      - 3P% / FT%
      - rimbalzi a partita (TRB.1)
      - assist a partita (AST.1)
      - punti a partita (PTS.1)
    Solo le parti coerenti con le skill richieste.
    """
    skills = intents.get("skills", [])
    name = player_row.get("Player", "Il giocatore")
    reasons = [f"{name} è stato selezionato perché:"]

    # Stato (Active / Retired)
    status_val = player_row.get("Status", None)
    if status_val == "Active":
        reasons.append("- è ancora in attività in NBA.")
    elif status_val == "Retired":
        reasons.append("- ha concluso la carriera NBA (giocatore ritirato).")

    # --- Percentuali tiro ---
    p3 = safe_get(player_row, "3P%")
    ft = safe_get(player_row, "FT%")

    # Shooting da 3
    if "shooting_3" in skills and p3 is not None:
        desc_3p = bucket_3p(p3)
        p3_str = format_percent_it(p3)
        if desc_3p and p3_str:
            reasons.append(f"- {desc_3p} ({p3_str} da tre in carriera).")

    # Tiri liberi
    if "ft" in skills and ft is not None:
        desc_ft = bucket_ft(ft)
        ft_str = format_percent_it(ft)
        if desc_ft and ft_str:
            reasons.append(f"- {desc_ft} ({ft_str} ai liberi in carriera).")

    # --- Rimbalzi totali ---
    if "reb" in skills:
        trb_total = safe_get(player_row, "TRB.1")
        if trb_total is not None:
            trb_str = format_int_it(trb_total)
            if trb_str:
                reasons.append(f"- ha catturato una media di {trb_str} rimbalzi a partita.")

    # --- Playmaking (assist totali) ---
    if "playmaking" in skills:
        ast_total = safe_get(player_row, "AST.1")
        if ast_total is not None:
            ast_str = format_int_it(ast_total)
            if ast_str:
                reasons.append(f"- è un buon creatore di gioco, con una media di{ast_str} assist a partita.")

    # --- Scorer (punti totali) ---
    if "scorer" in skills:
        pts_total = safe_get(player_row, "PTS.1")
        if pts_total is not None:
            pts_str = format_int_it(pts_total)
            if pts_str:
                reasons.append(f"- è un realizzatore di livello, con {pts_str} punti segnati a partita.")

    # Fallback se non ho ragioni specifiche oltre la prima riga
    if len(reasons) <= 1:
        reasons.append("- il suo profilo testuale è complessivamente molto simile alla descrizione richiesta.")

    explanation = "\n".join(reasons)
    return explanation


# ============================
# 7. Funzione principale: ranking + spiegazioni
# ============================

def rank_and_explain(query, top_k=5):
    base = search_players_scout(query, top_k)
    intents = base.attrs["intents"]

    # Merge con il dataset completo
    if "player_id" in base.columns and "player_id" in full_df.columns:
        merged = base.merge(full_df, on="player_id", how="left")
    else:
        merged = base.merge(full_df, on="Player", how="left")

    # FILTRO SU STATO (Active / Retired) SE RICHIESTO
    status_req = intents.get("status", None)  # "Active" / "Retired" / None
    if status_req is not None and "Status" in merged.columns:
        merged = merged[merged["Status"] == status_req]

    merged["explanation"] = merged.apply(lambda r: build_explanation(r, intents), axis=1)

    return merged, intents


# ============================
# 8. Test
# ============================

test_query = (
    "Cerco un tiratore con ottime percentuali da 3,  "
    "forte a rimbalzo e che sia ancora in attività."
)

results, intents = rank_and_explain(test_query, top_k=10)

print("Query originale:")
print(test_query)
print("\nIntenti interpretati:")
print(intents)

pd.set_option("display.max_colwidth", 200)

print("\nTOP risultati con spiegazioni (one-line):")
if "explanation" in results.columns:
    results["explanation_one_line"] = results["explanation"].str.replace("\n", " ")
    cols_to_show = [c for c in ["Player", "DraftYear", "Pick", "similarity", "explanation_one_line"] if c in results.columns]
    display(results[cols_to_show])
else:
    cols_to_show = [c for c in ["Player", "DraftYear", "Pick", "similarity"] if c in results.columns]
    display(results[cols_to_show])


Indice caricato.
TF-IDF matrix shape: (8323, 6183)
Metadata shape: (8323, 7)
Dataset stats shape: (8323, 36)
Query originale:
Cerco un tiratore con ottime percentuali da 3, buon tiratore di liberi, forte a rimbalzo e che sia ancora in attività.

Intenti interpretati:
{'normalized_query': 'cerco un tiratore con ottime percentuali da 3, buon tiratore di liberi, forte a rimbalzo e che sia ancora in attività.', 'skills': ['ft', 'reb', 'shooting_3'], 'status': 'Active'}

TOP risultati con spiegazioni (one-line):


,Player,similarity,explanation_one_line
0,Joseph Forte,0.289179,"Joseph Forte è stato selezionato perché: - tiratore da 3 punti poco affidabile (0,0% da tre in carriera). - buon tiratore di tiri liberi (80,0% ai liberi in carriera). - ha catturato una media di ..."
1,Pétur Guðmundsson,0.244029,"Pétur Guðmundsson è stato selezionato perché: - ottimo tiratore da 3 punti (50,0% da tre in carriera). - buon tiratore di tiri liberi (75,4% ai liberi in carriera). - ha catturato una media di 4 r..."
2,Ike Diogu,0.232326,"Ike Diogu è stato selezionato perché: - ottimo tiratore da 3 punti (50,0% da tre in carriera). - buon tiratore di tiri liberi (78,6% ai liberi in carriera). - ha catturato una media di 3 rimbalzi ..."
3,Jeff Ayres,0.230469,"Jeff Ayres è stato selezionato perché: - ottimo tiratore da 3 punti (40,0% da tre in carriera). - buon tiratore di tiri liberi (77,6% ai liberi in carriera). - ha catturato una media di 3 rimbalzi..."
4,Eric White,0.222536,"Eric White è stato selezionato perché: - ottimo tiratore da 3 punti (100,0% da tre in carriera). - buon tiratore di tiri liberi (79,8% ai liberi in carriera). - ha catturato una media di 2 rimbalz..."
5,Ron Rowan,0.221971,"Ron Rowan è stato selezionato perché: - ottimo tiratore da 3 punti (100,0% da tre in carriera). - buon tiratore di tiri liberi (75,0% ai liberi in carriera). - ha catturato una media di 0 rimbalzi..."
6,Igor Rakočević,0.219803,"Igor Rakočević è stato selezionato perché: - ottimo tiratore da 3 punti (41,7% da tre in carriera). - buon tiratore di tiri liberi (80,6% ai liberi in carriera). - ha catturato una media di 0 rimb..."
7,Brian Rowsom,0.215472,"Brian Rowsom è stato selezionato perché: - ottimo tiratore da 3 punti (66,7% da tre in carriera). - buon tiratore di tiri liberi (81,8% ai liberi in carriera). - ha catturato una media di 3 rimbal..."
8,Richard Petruška,0.214511,"Richard Petruška è stato selezionato perché: - ottimo tiratore da 3 punti (46,7% da tre in carriera). - buon tiratore di tiri liberi (75,0% ai liberi in carriera). - ha catturato una media di 1 ri..."
9,Mike Iuzzolino,0.213118,"Mike Iuzzolino è stato selezionato perché: - ottimo tiratore da 3 punti (40,4% da tre in carriera). - buon tiratore di tiri liberi (79,8% ai liberi in carriera). - ha catturato una media di 2 rimb..."


Ora il tuo sistema non fa solo:

“Ecco i giocatori simili alla query”.

ma fa:

“Ecco i giocatori simili e ti spiego perché ciascuno è in classifica.”

In pratica hai ottenuto:

Un ranking basato su IR + query pesata (Step C),

Un livello di explainable AI:

ruolo coerente con la richiesta,

statistiche compatibili con le skill richieste (3P%, FT%, TRB, ORB, AST, DWS, PTS),

frase in linguaggio naturale, pronta da mostrare a un talent scout / direttore sportivo.

Questo è perfetto per la presentazione: puoi dire che la Parte 2 non è solo un motore di ricerca, ma un vero assistente intelligente per lo scouting NBA, che non solo restituisce un ranking, ma giustifica anche le sue raccomandazioni. 🏀📊